In [1]:
import simplejson
import json
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import linecache

In [2]:
# Methods for counting lines
def _make_gen(reader):
    b = reader(1024 * 1024)
    while b:
        yield b
        b = reader(1024*1024)

def rawgencount(filename):
    f = open(filename, 'rb')
    f_gen = _make_gen(f.raw.read)
    return sum(buf.count(b'\n') for buf in f_gen)

# Source: https://stackoverflow.com/questions/845058/how-to-get-line-count-of-a-large-file-cheaply-in-python

In [3]:
rawgencount("league_ranked_2020_short.json")

10

In [6]:
test = linecache.getline("league_ranked_2020_short.json", 1 + 1).rstrip('\n')

In [9]:
lol = simplejson.loads(test)

In [10]:
simplejson.__version__

'3.17.5'

In [14]:
lol[2]["championId"]

41

In [23]:
championidtable_location = open("E:/Datasets/league_of_legends_ranked_games_2020/champion_and_items/riot_champion.csv", encoding='utf8')
championid_to_name = pd.read_csv(championidtable_location)
lookuptable = championid_to_name["key"].values.tolist()

In [24]:
line = linecache.getline("league_ranked_2020_short.json", 0 + 1).rstrip('\n')
json_line = simplejson.loads(line)

# Get the Champion keys of every player in one match
data = [json_line[0]["championId"] for i in range(10)]

# convert champion keys into index notation
data = torch.tensor([lookuptable.index(data[0]) for i in range(10)])

# convert data into one hot vecot encoding
data = torch.eye(148).index_select(dim=0, index=data)

target = torch.tensor([int(json_line[1]["stats"]["win"]),
                       int(json_line[8]["stats"]["win"])
                       ])

In [25]:
target

tensor([0, 1])

In [26]:
data

tensor([[0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.]])